In [13]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from imgaug import augmenters as iaa
from skimage.feature import hog
from sklearn.model_selection import train_test_split
import mediapipe as mp

In [14]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return
    
    # Get video details
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define the codec and create VideoWriter object
    out_filename = 'output_video.avi'
    out = cv2.VideoWriter(out_filename, cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_width, frame_height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert frame to RGB for Mediapipe processing
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detect faces and get landmarks
        results = face_mesh.process(image_rgb)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Extract landmarks
                landmarks = []
                for landmark in face_landmarks.landmark:
                    landmarks.extend([landmark.x, landmark.y, landmark.z])
                
                # Extract HOG features (if needed)
                g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                gray =cv2.resize(g, (48, 48))
                hog_features = extract_hog_features(gray)
                
                # Ensure HOG features are flattened
                hog_features = hog_features.flatten()
                
                # Concatenate HOG features and landmarks
                features = np.hstack([hog_features, landmarks])
                
                # Check if feature size matches expected by SVM model
                assert len(features) == 2304, f"Expected 2304 features, but got {len(features)} features."
                
                # Predict using SVM model
                predicted_label = svm_model.predict([features])[0]
                
                # Draw bounding box around face
                xmin, ymin, xmax, ymax = get_bounding_box(face_landmarks, frame_width, frame_height)
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                
                # Write label on the bounding box
                cv2.putText(frame, predicted_label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
        # Write the frame into the file 'output_video.avi'
        out.write(frame)
        
        # Display the resulting frame
        cv2.imshow('Emotion Prediction', frame)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release everything if job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [17]:
# Example usage: Replace 'video_path' with your video file path
video_path = r'D:\ITI\Graduation proj\Videos_data\output_video_79.mp4'
process_video(video_path)
